In [1]:
%matplotlib inline
import PartExtractor.DeformationField as DeformationField
import PartExtractor.DNerfManager as DNerfManager
from run_dnerf import config_parser, create_nerf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
from load_blender import pose_spherical
from run_dnerf import render_path
from run_dnerf_helpers import to8b
from scipy.spatial.transform import Rotation as R
import numpy as np
import cv2 as cv
import json

In [2]:
config_file = "configs/ball_cube_2.txt"
object_poses_yaml = "/home/stanlew/src/D-NeRF/data/BallCube_Take2/base_data/object_simulation_result.json"
manager = DNerfManager.DNerfManager(config_file = config_file)

num_frames = 60
times = torch.linspace(0.0,1.0,num_frames)#torch.Tensor([0.0, 0.25, 0.5, 0.75, 1.0])

NeRF type selected: direct_temporal
Found ckpts ['./logs/ball_cube_frame_fix_2/050000.tar', './logs/ball_cube_frame_fix_2/100000.tar', './logs/ball_cube_frame_fix_2/150000.tar', './logs/ball_cube_frame_fix_2/200000.tar', './logs/ball_cube_frame_fix_2/250000.tar', './logs/ball_cube_frame_fix_2/300000.tar', './logs/ball_cube_frame_fix_2/350000.tar', './logs/ball_cube_frame_fix_2/400000.tar', './logs/ball_cube_frame_fix_2/450000.tar', './logs/ball_cube_frame_fix_2/500000.tar', './logs/ball_cube_frame_fix_2/550000.tar', './logs/ball_cube_frame_fix_2/600000.tar', './logs/ball_cube_frame_fix_2/650000.tar', './logs/ball_cube_frame_fix_2/700000.tar', './logs/ball_cube_frame_fix_2/750000.tar', './logs/ball_cube_frame_fix_2/800000.tar', './logs/ball_cube_frame_fix_2/latest.tar']
Reloading from ./logs/ball_cube_frame_fix_2/latest.tar


In [3]:
object_poses_data = json.load(open(object_poses_yaml,'r'))
objects_names = ['object_cube','object_sphere']
obj_pose_dict = {}
for obj in objects_names:
    cur_obj_data = object_poses_data[obj]
    cur_obj_pose_dict = {}
    for i in range(num_frames):
        key = "frame_{0:02d}".format(i)
        time_key = times[i].item()
        pose = torch.Tensor(cur_obj_data[key])
        cur_obj_pose_dict[time_key] = pose
    obj_pose_dict[obj] = cur_obj_pose_dict

In [8]:
additional_samples = torch.tensor([[-0.25,0.25,0.5+0.2],[-0.25,0.25,0.5+0.1],[-0.25,0.25,0.5-0.2],[-0.25,0.25,0.5-0.1],]).unsqueeze(0)
field = DeformationField.DeformationField(manager,\
                                          times,\
                                          torch.Tensor([[-1.0,1.0],[-1.0,1.0],[-1.0,1.0]]),\
                                          torch.Tensor([20,20,20]).to(dtype=torch.int32),\
                                          additional_samples)

torch.Size([1, 8004, 3])
tensor([[[-1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -0.8947],
         [-1.0000, -1.0000, -0.7895],
         ...,
         [-0.2500,  0.2500,  0.6000],
         [-0.2500,  0.2500,  0.3000],
         [-0.2500,  0.2500,  0.4000]]])


In [19]:
field.samples[8001,:]

tensor([[[ 1.0000e+00,  0.0000e+00,  0.0000e+00, -2.4970e-01],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  2.4998e-01],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  6.0094e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 9.9935e-01,  3.4420e-02, -1.1092e-02, -2.5053e-01],
         [-3.5097e-02,  9.9706e-01, -6.8045e-02,  2.5062e-01],
         [ 8.7168e-03,  6.8390e-02,  9.9762e-01,  6.0014e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 9.8090e-01,  2.8835e-02, -1.9238e-01, -2.4961e-01],
         [-1.3120e-03,  9.8991e-01,  1.4168e-01,  2.5010e-01],
         [ 1.9453e-01, -1.3873e-01,  9.7104e-01,  6.0982e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 9.9969e-01,  2.5019e-04, -2.4955e-02, -2.4974e-01],
         [-3.0071e-04,  1.0000e+00, -2.0219e-03,  2.5015e-01],
         [ 2.4954e-02,  2.0288e-03,  9.9969e-01,  6.0312e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00, 

###### 